In [2]:
from langchain_community.document_loaders import WebBaseLoader

In [55]:
# function to check if url is responsive
from urllib.request import urlopen
from urllib.error import *
 
# try block to read URL
try:
    html = urlopen("https://colah.github.io/posts/2015-08-Understanding-LSTMs/")
     
except HTTPError as e:
    print("HTTP error", e)
     
except URLError as e:
    print("Opps ! Page not found!", e)
 
else:
    print('Yeah !  found ')

Yeah !  found 


In [51]:
## Load text from web
loader = WebBaseLoader("https://colah.github.io/posts/2015-08-Understanding-LSTMs/")
loader

In [52]:
## convert to documents
docs = loader.load()
docs

[Document(metadata={'source': 'https://colah.github.io/posts/2015-08-Understanding-LSTMs/', 'title': "Understanding LSTM Networks -- colah's blog", 'language': 'en'}, page_content="\n\n\n\n\nUnderstanding LSTM Networks -- colah's blog\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\ncolah's blog\n\n\n\n\n  Blog\n  About\n  Contact\n\n\n\n\n\n\n\n\nUnderstanding LSTM Networks\n\nPosted on August 27, 2015\n\n\n\nRecurrent Neural Networks\nHumans don’t start their thinking from scratch every second. As you read this essay, you understand each word based on your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Your thoughts have persistence.\nTraditional neural networks can’t do this, and it seems like a major shortcoming. For example, imagine you want to classify what kind of event is happening at every point in a movie. It’s unclear how a traditional neural network could use its reasoning about 

In [53]:
docs

[Document(metadata={'source': 'https://colah.github.io/posts/2015-08-Understanding-LSTMs/', 'title': "Understanding LSTM Networks -- colah's blog", 'language': 'en'}, page_content="\n\n\n\n\nUnderstanding LSTM Networks -- colah's blog\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\ncolah's blog\n\n\n\n\n  Blog\n  About\n  Contact\n\n\n\n\n\n\n\n\nUnderstanding LSTM Networks\n\nPosted on August 27, 2015\n\n\n\nRecurrent Neural Networks\nHumans don’t start their thinking from scratch every second. As you read this essay, you understand each word based on your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Your thoughts have persistence.\nTraditional neural networks can’t do this, and it seems like a major shortcoming. For example, imagine you want to classify what kind of event is happening at every point in a movie. It’s unclear how a traditional neural network could use its reasoning about 

In [50]:
## text split into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
split = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = split.split_documents(docs)
documents

[Document(metadata={'source': 'https://colah.github.io/posts/2015-08-Understanding-LSTs/', 'title': 'Page not found · GitHub Pages', 'language': 'No language found.'}, page_content='Page not found · GitHub Pages\n\n\n\n\n404\nFile not found\n\n        The site configured at this address does not\n        contain the requested file.\n      \n\n        If this is your site, make sure that the filename case matches the URL\n        as well as any file permissions.\n        For root URLs (like http://example.com/) you must provide an\n        index.html file.\n      \n\nRead the full documentation\n        for more information about using GitHub Pages.\n      \n\nGitHub Status —\n        @githubstatus')]

In [30]:
## import Ollama Embeddings

from langchain_community.embeddings import OllamaEmbeddings

embeddings = (OllamaEmbeddings(model='gemma:2b'))

In [31]:
from langchain_community.vectorstores import FAISS

vc_store = FAISS.from_documents(documents,embeddings)

In [32]:
## similarity from vectorstore based on embeddings

query = "Disadvantage of RNN compared to LSTM are as follows :"
vc_store.similarity_search(query)

[Document(metadata={'source': 'https://colah.github.io/posts/2015-08-Understanding-LSTMs/', 'title': "Understanding LSTM Networks -- colah's blog", 'language': 'en'}, page_content='In theory, RNNs are absolutely capable of handling such “long-term dependencies.” A human could carefully pick parameters for them to solve toy problems of this form. Sadly, in practice, RNNs don’t seem to be able to learn them. The problem was explored in depth by Hochreiter (1991) [German] and Bengio, et al. (1994), who found some pretty fundamental reasons why it might be difficult.\nThankfully, LSTMs don’t have this problem!\nLSTM Networks\nLong Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by Hochreiter & Schmidhuber (1997), and were refined and popularized by many people in following work.1 They work tremendously well on a large variety of problems, and are now widely used.\nLSTMs are explicitly des

In [12]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="gemma:2b")

In [13]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [14]:
## Prompt to interact with LLM
prompt1=ChatPromptTemplate.from_template(
"""Question: {question}

Answer: Let's think step by step."""
)

In [15]:
chain = prompt1 | llm

In [16]:
chain.invoke({"question":query})

"Sure, here's a step-by-step analysis of the disadvantages of RNN compared to LSTM:\n\n**1. Memory Efficiency:**\n- LSTM has a longer memory span due to its ability to access previous inputs.\n- RNNs have a limited memory capacity, which can lead to issues when dealing with long sequences.\n\n**2. Gradient Vanishing/Exploding:**\n- LSTMs are less susceptible to gradient vanishing or exploding compared to RNNs.\n- RNNs can suffer from these issues when dealing with sequential data, where the earlier parts of the sequence are less relevant than the later parts.\n\n**3. Training Complexity:**\n- LSTMs are more complex to train than RNNs due to their attention mechanism.\n- The attention mechanism requires calculating the dot product between all pairs of input and output sequences, which can be computationally expensive.\n\n**4. Limited Attention Mechanism:**\n- RNNs have a simpler attention mechanism compared to LSTMs.\n- The attention weights in RNNs are calculated based on the dot produ

In [17]:
from langchain.chains import create_retrieval_chain

In [23]:
prompt2 = ChatPromptTemplate.from_template(
"""
Answer the following question based on the provided context. You can use outside resource but try to answer with respect to the given context. If you use other resource then mention it as well:
<context>
{context}
</context>
"""
)
chain2 = create_stuff_documents_chain(llm,prompt2)
retriever2 = vc_store.as_retriever()
retrieval_chain2 = create_retrieval_chain(retriever2,chain2)

In [25]:
result = retrieval_chain2.invoke({"input":query})
result['answer']

'According to the context, RNNs struggle to learn long-term dependencies due to the lack of a mechanism that explicitly allows them to remember past information. LSTMs are specifically designed to avoid this problem by explicitly remembering past information in the cell state.'

In [26]:
# points from text that helped to arrive at the conclusion
result['context']

[Document(metadata={'source': 'https://colah.github.io/posts/2015-08-Understanding-LSTMs/', 'title': "Understanding LSTM Networks -- colah's blog", 'language': 'en'}, page_content='In theory, RNNs are absolutely capable of handling such “long-term dependencies.” A human could carefully pick parameters for them to solve toy problems of this form. Sadly, in practice, RNNs don’t seem to be able to learn them. The problem was explored in depth by Hochreiter (1991) [German] and Bengio, et al. (1994), who found some pretty fundamental reasons why it might be difficult.\nThankfully, LSTMs don’t have this problem!\nLSTM Networks\nLong Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by Hochreiter & Schmidhuber (1997), and were refined and popularized by many people in following work.1 They work tremendously well on a large variety of problems, and are now widely used.\nLSTMs are explicitly des